In [1]:
%cd F:\Text to Speech\vits


F:\Text to Speech\vits


In [2]:
!gdown 'https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT'

Access denied with the following error:


 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id='https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT' 



In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(arg=0, lineno=1042)
          10	LOAD_CONST(arg=2, lineno=1042)
          12	BINARY_SUBSCR(arg=None, lineno=1042)
          14	COMPARE_OP(arg=4, lineno=1042)
          16	LOAD_FAST(arg=0, lineno=1042)
          18	LOAD_CONST(arg=1, lineno=1042)
          20	BINARY_SUBSCR(arg=None, lineno=1042)
          22	LOAD_FAST(arg=0, lineno=1042)
          24	LOAD_CONST(arg=3, lineno=1042)
          26	BINARY_SUBSCR(arg=None, lineno=1042)
          28	COMPARE_OP(arg=5, lineno=1042)
          30	BINARY_AND(arg=None, lineno=1042)
          32	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

In [4]:
hps = utils.get_hparams_from_file(r"F:\Text to Speech\vits\configs\ljs_base.json")
hps_ms = utils.get_hparams_from_file(r"F:\Text to Speech\vits\configs\vctk_base.json") 

In [9]:
!gdown 'https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru'

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id='https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru' 



In [10]:
net_g_ms = SynthesizerTrn(
    len(symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_vctk.pth", net_g_ms, None)

INFO:root:Loaded checkpoint 'pretrained_vctk.pth' (iteration 0)


In [5]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_ljs.pth", net_g, None)

INFO:root:Loaded checkpoint 'pretrained_ljs.pth' (iteration 0)


In [7]:
from mel_processing import spectrogram_torch
from utils import load_wav_to_torch

audio, sampling_rate = load_wav_to_torch(r"F:\Text to Speech\vits\vc_01_01.wav")
        
y = audio / hps_ms.data.max_wav_value
y = y.unsqueeze(0)

In [26]:
y.shape

torch.Size([1, 65693])

In [8]:
spec = spectrogram_torch(y, hps_ms.data.filter_length,
    hps_ms.data.sampling_rate, hps_ms.data.hop_length, hps_ms.data.win_length,
    center=False)
spec_lengths = torch.LongTensor([spec.size(-1)])
sid_src = torch.LongTensor([81])

In [31]:
print(y.shape)
print(spec.shape)


torch.Size([1, 65693])
torch.Size([1, 513])


In [32]:
audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.float().numpy()

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [35]:
with torch.no_grad():
    sid_tgt1 = torch.LongTensor([0])
    sid_tgt2 = torch.LongTensor([14])
    sid_tgt3 = torch.LongTensor([1])
    sid_tgt4 = torch.LongTensor([17])
    audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1).data.float().numpy()
    audio2 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.float().numpy()
    audio3 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.float().numpy()
    audio4 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt4)[0][0,0].data.float().numpy()

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [30]:
sid_tgt1.shape

torch.Size([1])

In [33]:
with torch.no_grad():
    sid_tgt1 = torch.LongTensor([77])
    sid_tgt2 = torch.LongTensor([14])
    sid_tgt3 = torch.LongTensor([1])
    sid_tgt4 = torch.LongTensor([17])
    audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.float().numpy()
    audio2 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.float().numpy()
    audio3 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.float().numpy()
    audio4 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt4)[0][0,0].data.float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].numpy(), rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt4.item())
ipd.display(ipd.Audio(audio4, rate=hps_ms.data.sampling_rate))

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)